In [1]:
import requests
import pandas as pd
from lookups import SourcesAPI
from io import StringIO

In [2]:
response = requests.get(SourcesAPI.PER_CAPITA_AUSTIN_INCOME.value)
if response.status_code == 200:
    df = pd.read_csv(StringIO(response.text))
else:
    print(f'Failed to download data. Status code: {response.status_code}')

In [7]:
df

,DATE,PCPI48015
0,1969-01-01,2852
1,1970-01-01,3259
2,1971-01-01,3701
3,1972-01-01,4239
4,1973-01-01,4421
5,1974-01-01,5213
6,1975-01-01,5943
7,1976-01-01,6724
8,1977-01-01,7051
9,1978-01-01,8011


In [29]:
df.iloc[:,[1]]

,PCPI48015
0,39514
1,40214
2,42000
3,47474
4,46616
5,49701
6,50727
7,48332
8,49081
9,49990


In [6]:
import requests
import pandas as pd
from io import StringIO

# URL for downloading the CSV data
download_url = 'https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1138&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=PCPI48113&scale=left&cosd=1969-01-01&coed=2021-01-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Annual&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2023-10-07&revision_date=2023-10-07&nd=1969-01-01'

# Send a GET request to the download URL
response = requests.get(download_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Read the CSV data into a pandas DataFrame
    df = pd.read_csv(StringIO(response.text))
else:
    print(f'Failed to download data. Status code: {response.status_code}')

In [8]:
df

,DATE,PCPI48113
0,1969-01-01,4445
1,1970-01-01,4670
2,1971-01-01,4928
3,1972-01-01,5428
4,1973-01-01,5956
5,1974-01-01,6557
6,1975-01-01,7066
7,1976-01-01,7607
8,1977-01-01,8363
9,1978-01-01,9554
